# 8-6. 프로젝트 : 커스텀 프로젝트 직접 만들기
- 2024.8.26
- 구태훈

In [ ]:
!pip uninstall -y datasets pyarrow transformers tensorflow

Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Successfully uninstalled pyarrow-14.0.2
Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0


In [ ]:
!pip install datasets==2.14.5 pyarrow==14.0.1 transformers tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 57.2 MB/s eta 0:00:00
Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 184.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:0

In [ ]:
import tensorflow as tf
import numpy as np
import transformers
import argparse
import tensorflow_datasets as tfds


print(tf.__version__)
print(np.__version__)
print(transformers.__version__)
print(argparse.__version__)

# 기존 버전 2.17.0 1.26.4 4.42.4 1.1



2.17.0
1.26.4
4.44.2
1.1


STEP 1. mnli 데이터셋을 분석해 보기
- tensorflow-datasets를 이용하여 glue/mnli를 다운로드하려면 tensorflow-datasets 라이브러리 버전을 올려야 합니다.



In [ ]:
!pip install tensorflow-datasets -U

In [ ]:
# MNLI 데이터셋 로드
mnli_dataset, mnli_info = tfds.load('glue/mnli', with_info=True)

print(mnli_info)

# 훈련 데이터 예시 확인
for example in mnli_dataset['train'].take(1):
    print(example)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/5 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

Generating test_matched examples...:   0%|          | 0/9796 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/glue/mnli/incomplete.IJIFIN_2.0.0/glue-test_matched.tfrecord*...:   0%|   …

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/glue/mnli/incomplete.IJIFIN_2.0.0/glue-test_mismatched.tfrecord*...:   0%|…

Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='glue',
    full_name='glue/mnli/2.0.0',
    description="""
    GLUE, the General Language Understanding Evaluation benchmark
    (https://gluebenchmark.com/) is a collection of resources for training,
    evaluating, and analyzing natural language understanding systems.
    """,
    config_description="""
    The Multi-Genre Natural Language Inference Corpus is a crowdsourced
    collection of sentence pairs with textual entailment annotations. Given a premise sentence
    and a hypothesis sentence, the task is to predict whether the premise entails the hypothesis
    (entailment), contradicts the hypothesis (contradiction), or neither (neutral). The premise sentences are
    gathered from ten different sources, including transcribed speech, fiction, and government reports.
    We use the standard test set, for which we obtained pri

STEP 2. MNLIProcessor클래스 구현하기


In [ ]:
from transformers import DataProcessor, InputExample

class MNLIProcessor(DataProcessor):
    def get_train_examples(self, data_dir):
        return self._create_examples(mnli_dataset['train'], "train")

    def get_dev_examples(self, data_dir):
        return self._create_examples(mnli_dataset['validation_matched'], "dev_matched")

    def get_test_examples(self, data_dir):
        return self._create_examples(mnli_dataset['test_matched'], "test_matched")

    def get_labels(self):
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, dataset, set_type):
        examples = []
        for (i, example) in enumerate(dataset):
            guid = f"{set_type}-{i}"
            text_a = example['premise'].numpy().decode('utf-8')
            text_b = example['hypothesis'].numpy().decode('utf-8')
            if set_type != "test_matched":
                label = self.get_labels()[example['label'].numpy()]
            else:
                label = "contradiction"
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

STEP 3. 위에서 구현한 processor 및 Huggingface에서 제공하는 tokenizer를 활용하여 데이터셋 구성하기


- RoBERTa 모델 사용

In [ ]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
processor = MNLIProcessor()

def convert_examples_to_features(examples, tokenizer, max_length=128):
    features = []
    for (ex_index, example) in enumerate(examples):
        inputs = tokenizer.encode_plus(
            example.text_a,
            example.text_b,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True
        )

        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        features.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
            'label': example.label
        })

    return features

train_examples = processor.get_train_examples(None)[:5000]
train_features = convert_examples_to_features(train_examples, tokenizer)

val_examples = processor.get_dev_examples(None)[:1000]
val_features = convert_examples_to_features(val_examples, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the return

STEP 4. model을 생성하여 학습 및 테스트를 진행해 보기


In [ ]:
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# 데이터를 tf.data.Dataset으로 변환
def create_dataset(features):
    return tf.data.Dataset.from_tensor_slices(({
        'input_ids': [f['input_ids'] for f in features],
        'attention_mask': [f['attention_mask'] for f in features],
        'token_type_ids': [f['token_type_ids'] for f in features]
    }, [processor.get_labels().index(f['label']) for f in features])).batch(32)

train_dataset = create_dataset(train_features)
val_dataset = create_dataset(val_features)

# 모델 학습
history = model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# 테스트 데이터로 평가
test_examples = processor.get_test_examples(None)[:500]
test_features = convert_examples_to_features(test_examples, tokenizer)
test_dataset = create_dataset(test_features)

results = model.evaluate(test_dataset)
print(f"Test loss: {results[0]}")
print(f"Test accuracy: {results[1]}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/2
157/157 [==============================] - 174s 882ms/step - loss: 0.8922 - accuracy: 0.5764 - val_loss: 0.5652 - val_accuracy: 0.7760
Epoch 2/2
16/16 [==============================] - 4s 264ms/step - loss: 2.2774 - accuracy: 0.3360
Test loss: 2.2774322032928467
Test accuracy: 0.335999995470047


In [ ]:
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# 데이터를 tf.data.Dataset으로 변환
def create_dataset(features):
    return tf.data.Dataset.from_tensor_slices(({
        'input_ids': [f['input_ids'] for f in features],
        'attention_mask': [f['attention_mask'] for f in features],
        'token_type_ids': [f['token_type_ids'] for f in features]
    }, [processor.get_labels().index(f['label']) for f in features])).batch(32)

train_dataset = create_dataset(train_features)
val_dataset = create_dataset(val_features)

# 모델 학습
history = model.fit(train_dataset, epochs=3, validation_data=val_dataset)

# 테스트 데이터로 평가
test_examples = processor.get_test_examples(None)
test_features = convert_examples_to_features(test_examples, tokenizer)
test_dataset = create_dataset(test_features)

results = model.evaluate(test_dataset)
print(f"Test loss: {results[0]}")
print(f"Test accuracy: {results[1]}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/3
157/157 [==============================] - 174s 882ms/step - loss: 0.8880 - accuracy: 0.5594 - val_loss: 0.5416 - val_accuracy: 0.7810
Epoch 2/3
157/157 [==============================] - 133s 847ms/step - loss: 0.5303 - accuracy: 0.7970 - val_loss: 0.5846 - val_accuracy: 0.7870
Epoch 3/3
157/157 [==============================] - 132s 840ms/step - loss: 0.3631 - accuracy: 0.8768 - val_loss: 0.5746 - val_accuracy: 0.7990


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

307/307 [==============================] - 83s 270ms/step - loss: 2.7700 - accuracy: 0.3210
Test loss: 2.770005464553833
Test accuracy: 0.32104942202568054


# 회고

- 허깅 페이스에서 모델을 가져오는 방식이 상대적으로 간편하게 느껴졌습니다.
- 클래스와 함수를 이해하는데 아직 어려움이 있어서 직접 작성한 부분은 매우 적었습니다.
- 데이터셋에 맞는 인스턴스를 생성하고 학습하는 과정이 추후 필요시 재학습이 필요합니다.
- epochs가 2일때와 3일때 두번을 진행했는데 학습과 테스트 큰 차이 없이 모자란 수치를 보여줬습니다.
- 코랩에서 실행을 했습니다. gpu의 자원이 모자람을 많이 느꼈습니다. 로컬 환경에서 작업해야함을 느꼈습니다.